In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [5]:
df = pd.read_csv('/kaggle/input/london-bike-sharing-dataset/london_merged.csv', parse_dates=['timestamp'])
df.head()

- 구조모형으로 습도, 온도, 시간 등의 변수들을 넣어서 자전거 이용객 수를 예측하는 모형을 만들어 보자

In [9]:
# 데이터의 타입과 구조

print('데이터의 타입은:', df.dtypes)
print("데이터의 구조는:", df.shape)
print('데이터의 컬럼은:', df.columns)

In [11]:
# 결측치 확인

df.isna().sum()

In [12]:
# 시각화를 통해 결측치 확인

msno.matrix(df)
plt.show()

In [13]:
# timestamp 개별적으로 추출하기

df['year'] = df['timestamp'].dt.year #연도 추출
df['month'] = df['timestamp'].dt.month #월 추출
df['dayofweek'] = df['timestamp'].dt.dayofweek #월화수목금토일 추출 #번호
df['hour'] = df['timestamp'].dt.hour #시각 추출

df.head()

In [14]:
# 탐색적 분석

df['year'].value_counts() #year의 구성을 확인

In [15]:
df['month'].value_counts() #month의 구성을 확인

In [16]:
df['dayofweek'].value_counts() #dayofweek의 구성을 확인

In [17]:
df['weather_code'].value_counts() #weather_code의 구성을 확인 #범주형

In [21]:
# 그래프 그리기

a, b = plt.subplots(1, 1, figsize = (10, 5))
sns.boxplot(df['year'], df['cnt'])

In [22]:
a, b = plt.subplots(1, 1, figsize = (10, 5))
sns.boxplot(df['month'], df['cnt'])

In [23]:
a, b = plt.subplots(1, 1, figsize = (10, 5))
sns.boxplot(df['dayofweek'], df['cnt'])

In [24]:
a, b = plt.subplots(1, 1, figsize = (10, 5))
sns.boxplot(df['hour'], df['cnt'])

- 출퇴근 시간에 자전거를 많이 이용하는 모습 확인 할 수 있음

In [27]:
# 그래프 함수 만들기

def plot_bar(data, feature):
    fig = plt.figure(figsize=(12,3))
    sns.barplot(x=feature, y='cnt', data=data, palette='Set3', orient='v')

In [28]:
plot_bar(df, 'hour')

In [29]:
plot_bar(df, 'dayofweek')

In [30]:
# 아웃라이어 제거, 카테고리형 변수로 변환, 훈련/테스트 데이터 분리 

# 아웃라이어 제거

def is_outlier(s): #시그마 이상치 제거 방법
    lower_limit = s.mean() - (s.std()*3) #평균에서 표준편차의 3배 만큼 뺀 값
    upper_limit = s.mean() + (s.std()*3)
    return ~s.between(lower_limit, upper_limit) #상한값과 하한값 사이의 값을 리턴

In [32]:
df_out = df[~df.groupby('hour')['cnt'].apply(is_outlier)] #시간대별 자전거 이용객수를 감안하여 이상치 제거

print('이상치 제거전:', df.shape)
print('이상치 제거후:', df_out.shape)

- 이상치로 150 정도가 제거되었음

In [33]:
df_out.dtypes

- 특정 숫자가 의미/내용을 가지고있는 데이터들은 카테고리컬 변수들로 변환 시켜야 함

In [34]:
# 타입 변환
df_out['weather_code'] = df_out['weather_code'].astype('category') #astype -> 원하는 타입으로 변환
df_out['season'] = df_out['season'].astype('category')
df_out['year'] = df_out['year'].astype('category')
df_out['month'] = df_out['month'].astype('category')
df_out['hour'] = df_out['hour'].astype('category')

In [35]:
df_out.dtypes

In [36]:
# 더미 변수 처리 -> 머신러닝은 안해도 큰 영향을 안미치지만, 딥러닝의 경우는 꼭 해줘야 함
# 이진 숫자 처리

df_out = pd.get_dummies(df_out, columns=['weather_code', 'season', 'year', 'month', 'hour'])
df_out.head()

In [37]:
df_out.shape

최종적으로 우리가 예측 하고자 하는건
- 시간대별 자전거 수요 이용객
    - 우리가 예측하고자 하는 것 = 종속변수 = y = cnt
    - 그 외 나머지 = 독립변수 = x 

In [39]:
# 종속변수와 독립변수 분리

df_y = df_out['cnt']
df_x = df_out.drop(['timestamp', 'cnt'], axis=1) #axis=1: 열기준으로 데이터 버려줘
# timestamp는 이제 필요 없음

df_x.head()

In [40]:
df_y.head()

In [42]:
# 훈련용/테스트용 데이터 분리 

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state=66, test_size=0.3, shuffle=False ) #shuffle=False: 이 데이터는 시계열 데이터라서 데이터를 섞으면 안됨

In [43]:
print('x_train의 구조는:', x_train.shape)
print('y_train의 구조는:', y_train.shape)

print('x_test의 구조는:', x_test.shape)
print('y_test의 구조는:', y_test.shape)

In [44]:
# 딥러닝

import keras
from keras.models import Sequential #층을 쌓아서 하는 방식
from keras.layers import Dense
from keras.callbacks import EarlyStopping #과적합이 되지 않도록 잡아주는 라이브러리

In [46]:
model = Sequential()
model.add(Dense(units=160, activation='relu', input_dim=57)) #imput_dim은 독립변수 개수
model.add(Dense(units=60, activation='relu')) #층 추가
model.add(Dense(units=20, activation='relu')) #층 추가
model.add(Dense(units=1, activation='linear')) #시간대별 자건거 수요를 예측하는 것이라 units=1,  linear 이용

In [47]:
model.summary()

In [50]:
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
early_stopping = EarlyStopping(monitor='loss', patience=5, mode='min') #과적합이 될 때 loss가 5번 정도 올라가면 멈추라는 코드
history = model.fit(x_train, y_train, epochs=50, batch_size=1, validation_split=0.1, callbacks=[early_stopping]) 
#훈련횟수: 전체 데이터의 50퍼 정도 반복해줘, 훈련할 때 데이터를 배치사이즈가 1 단위로 할 거야
#validation_split: 훈련을 하면서 잘못된 방향으로 훈련이 되지 않도록(과적합 등) 10%를 검증용 데이터로 분리하여, 매 epochs가 돌 때마다 훈련이 잘 되고 있는지 체크하기 위한 검증용 데이터 분리 옵션
#과적합 문제를 낮추기 위해

- loss는 계속 줄어드는 추세
- val_loss는 중간에 올라가는 등의 추세를 보임. 만약 올라가는 모습이 5번 보이면 훈련을 멈추도록 했음

In [51]:
# 이 훈련 과정을 시각화해서 살펴보기

plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend(['val_loss', 'loss'])
plt.show()

- 학습 횟수에 따라 훈련용 데이터의 loss가 감소
- val_loss 또한 전반적으로 감소

In [52]:
# 예측
y_predict = model.predict(x_test)

In [55]:
# 머신러닝

# 회귀트리유형

from sklearn.metrics import mean_squared_error #mse: 실제값과 예측값의 차이 #rmse: mse의 루트를 씌운 값

def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))

In [56]:
# 앞에서 학습한 딥러닝의 RMSE 구해보기
print('RMSE', RMSE(y_test, y_predict))

- RMSE가 낮을수록 좋음

In [57]:
# 머신러닝_랜덤포레스트

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=16) #약한 100개의 모형 만들겠어
rf.fit(x_train, y_train)
rf_result = rf.predict(x_test)

print('RMSE', RMSE(y_test, rf_result))

- 현재 데이터로는 딥러닝보다 성능이 떨어지는 상황

In [58]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=100, random_state=16)
xgb.fit(x_train, y_train)
xgb_result = xgb.predict(x_test)

print('RMSE', RMSE(y_test, xgb_result))

- 랜덤 포레스트보다 성능이 떨어짐
- 머신러닝은 파라미터 설정의 영향을 많이 받으므로 파라미터에 따라 성능이 우위가 변할 수 있음

In [59]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor(n_estimators=100, random_state=16)
lgb.fit(x_train, y_train)
lgb_result = lgb.predict(x_test)

print('RMSE', RMSE(y_test, lgb_result))

- 어머 성능이 넘 안좋아,,,근데 일반적인 경우 lgb의 성능이 다른 두 머신러닝 모델보다 좋게나옴

In [61]:
# 모형별 결과 비교하기

# 앞에서 살펴본 총 4개의 모형으로 나온 결과들을 하나의 데이터 프레임으로 합친 후 그래프를 그려보자

lgb = pd.DataFrame(lgb_result)
xgb = pd.DataFrame(xgb_result)
rf = pd.DataFrame(rf_result)
dnn = pd.DataFrame(y_predict)
compare = pd.DataFrame(y_test).reset_index(drop=True) #y_test

compare.head() #y_test 옆에 이제 xgb, rf, dnn 추가할 것

In [63]:
compare['lgb'] = lgb
compare['xgb'] = xgb
compare['rf'] = rf
compare['dnn'] = dnn

compare.head() # cnt: 실제값, 나머지는 모델을 통한 예측값

In [67]:
# 그래프 비교

sns.kdeplot(compare['cnt'], shade=True, color='r')
sns.kdeplot(compare['xgb'], shade=True, color='b')
sns.kdeplot(compare['rf'], shade=True, color='y')
sns.kdeplot(compare['dnn'], shade=True, color='g')
sns.kdeplot(compare['lgb'], shade=True )
plt.legend(['cnt', 'xgb', 'rf', 'dnn', 'lgb'])